In [15]:
import pandas as pd
import numpy as np
import re

In [16]:
df=pd.read_csv("Luxury_Housing_Bangalore.csv")
df.describe()

,Unit_Size_Sqft,Connectivity_Score,Amenity_Score,Locality_Infra_Score,Avg_Traffic_Time_Min
count,90954.000000,101000.000000,90910.000000,101000.000000,101000.000000
mean,5972.274765,6.992619,7.503663,7.498426,67.182921
std,1783.397836,1.731757,1.440758,1.443128,30.271611
min,-1.000000,4.000031,5.000224,5.000013,15.000000
25%,4477.250000,5.494526,6.260329,6.247954,41.000000
50%,5990.000000,6.985805,7.499123,7.495614,67.000000
75%,7497.000000,8.490000,8.752207,8.749824,93.000000
max,8999.000000,9.999970,9.999865,9.999956,119.000000


In [17]:
df["Configuration"]=df["Configuration"].astype(str).str.upper()
df.head(10)

,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4BHK,12.750846039118798,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,yes,9.212491,18,Loved the amenities!
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3BHK,16.292151871065954,Primary,Other,2024-06-30,4.839024,NaN,Under construction,NRI Desk,no,7.723898,106,NaN
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4BHK,10.517724412961911,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,yes,6.985493,113,Agent was not responsive.
3,PROP000004,bellary road,Project_3,Embassy,6192.0,3BHK,9.396367494232896,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,yes,6.100929,106,Excellent location!
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4BHK,15.345392444511946,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,no,5.312510,18,Too far from my office.
5,PROP000006,YELAHANKA,Project_5,Brigade,4290.0,4BHK,₹9.82 Cr,Secondary,HNI,2023-09-30,7.936068,NaN,Launch,Online,no,5.154027,16,Underpriced for location.
6,PROP000007,Bannerghatta Road,Project_6,Prestige,6776.0,5BHK+,10.896752354445224,Secondary,Other,2025-03-31,8.643117,8.355238,Under construction,Direct,no,9.791550,74,Too far from my office.
7,PROP000008,whitefield,Project_7,Total Environment,5763.0,3BHK,9.357538943683888,Secondary,NRI,2025-03-31,6.781306,9.565013,Under construction,Direct,yes,9.054713,31,NaN
8,PROP000009,sarjapur road,Project_8,Prestige,5815.0,3BHK,12.299449409872416,Secondary,Other,2024-03-31,7.284164,5.580442,Under construction,NRI Desk,yes,8.552046,68,NaN
9,PROP000010,JP NAGAR,Project_9,Puravankara,8675.0,4BHK,9.193310278863123,Primary,CXO,2023-09-30,9.964399,7.759907,Launch,Direct,yes,8.395691,18,Too far from my office.


In [18]:
def clean_price(value):
    if pd.isna(value):
        return np.nan
    value =str(value).strip().replace(',','').lower()
    value=value.replace("₹","").replace("inr","").replace("rs.","").replace("rs","").strip()
    if "lakh" in value:
        num=re.findall(r"\d+\.?\d",value)
        return float(num[0])/100 if num else np.nan
    elif "cr" in value:
        num=re.findall(r"\d+\.?\d",value)
        return float(num[0])if num else np.nan
    elif "crore" in value:
        num=re.findall(r"\d+\.?\d",value)
        return float(num[0])if num else np.nan
    elif value.replace(".","",1).isdigit():
        return float(value)
    return np.nan
    

In [19]:
df["Ticket_Price_Cr"]=df["Ticket_Price_Cr"].apply(clean_price)

In [20]:
df["Amenity_Score"].describe

<bound method NDFrame.describe of 0         5.462863
1              NaN
2         8.669227
3         5.720246
4         8.609649
            ...   
100995    6.221131
100996    9.783611
100997    8.383013
100998    5.756247
100999    9.989290
Name: Amenity_Score, Length: 101000, dtype: float64>

In [21]:
df["Ticket_Price_Cr"]=df.groupby(["Configuration","Micro_Market"])["Ticket_Price_Cr"].transform(lambda x:x.fillna(x.median()))
df["Unit_Size_Sqft"]=df.groupby(["Configuration","Micro_Market"])["Unit_Size_Sqft"].transform(lambda x:x.fillna(x.median()))
df["Amenity_Score"]=df.groupby("Micro_Market")["Amenity_Score"].transform(lambda x:x.fillna(x.median()))
df["Buyer_Comments"]=df["Buyer_Comments"].fillna("No Comments")
df.isnull().sum()


Property_ID             0
Micro_Market            0
Project_Name            0
Developer_Name          0
Unit_Size_Sqft          0
Configuration           0
Ticket_Price_Cr         0
Transaction_Type        0
Buyer_Type              0
Purchase_Quarter        0
Connectivity_Score      0
Amenity_Score           0
Possession_Status       0
Sales_Channel           0
NRI_Buyer               0
Locality_Infra_Score    0
Avg_Traffic_Time_Min    0
Buyer_Comments          0
dtype: int64

In [22]:
text_columns=["Micro_Market","Developer_Name","Possession_Status","Sales_Channel","Buyer_Comments"]
for col in text_columns:
    df[col]=df[col].astype(str).str.strip().str.title()

##price per sqft
df["Price_per_Sqft"]=df["Ticket_Price_Cr"]*10000000/df["Unit_Size_Sqft"]


df["NRI_Buyer"] = df["NRI_Buyer"].astype(str).str.strip().str.lower().map({"yes": 1, "no": 0})

In [23]:
df["Purchase_Quarter"]=pd.to_datetime(df["Purchase_Quarter"])
df["Purchase_Year"] = df["Purchase_Quarter"].dt.year
df["Quarter_Number"] = df["Purchase_Quarter"].dt.quarter
df['Quarter_Label'] = df['Purchase_Quarter'].dt.to_period('Q').astype(str)

In [24]:
# 1. Booking Status (Primary = Booked)
df["Booking_Status"] = df["Transaction_Type"].apply(
    lambda x: "Booked" if x == "Primary" else "Not Booked"
)

# 2. Booking Count (numeric flag)
df["Booking_Count"] = df["Transaction_Type"].apply(
    lambda x: 1 if x == "Primary" else 0
)

# 3. Booking Conversion Rate (per Micro Market)
df["Booking_Conversion_Rate"] = df.groupby("Micro_Market")["Booking_Count"].transform(
    lambda x: round(x.sum() / len(x), 3)
)


In [25]:
cols_to_drop = ["Purchase_Year", "Quarter_Number", "Positive_Flag"]

# Drop only if they exist (safe drop)
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

In [26]:
df.columns

Index(['Property_ID', 'Micro_Market', 'Project_Name', 'Developer_Name',
       'Unit_Size_Sqft', 'Configuration', 'Ticket_Price_Cr',
       'Transaction_Type', 'Buyer_Type', 'Purchase_Quarter',
       'Connectivity_Score', 'Amenity_Score', 'Possession_Status',
       'Sales_Channel', 'NRI_Buyer', 'Locality_Infra_Score',
       'Avg_Traffic_Time_Min', 'Buyer_Comments', 'Price_per_Sqft',
       'Quarter_Label', 'Booking_Status', 'Booking_Count',
       'Booking_Conversion_Rate'],
      dtype='object')

In [27]:
df.head()

,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,...,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Price_per_Sqft,Quarter_Label,Booking_Status,Booking_Count,Booking_Conversion_Rate
0,PROP000001,Sarjapur Road,Project_0,Rmz,4025.0,4BHK,12.750846,Primary,NRI,2025-03-31,...,Broker,1,9.212491,18,Loved The Amenities!,31679.120594,2025Q1,Booked,1,0.494
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3BHK,16.292152,Primary,Other,2024-06-30,...,Nri Desk,0,7.723898,106,No Comments,28284.985887,2024Q2,Booked,1,0.508
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4BHK,10.517724,Primary,HNI,2023-12-31,...,Direct,1,6.985493,113,Agent Was Not Responsive.,13646.976013,2023Q4,Booked,1,0.506
3,PROP000004,Bellary Road,Project_3,Embassy,6192.0,3BHK,9.396367,Primary,HNI,2024-03-31,...,Online,1,6.100929,106,Excellent Location!,15175.012103,2024Q1,Booked,1,0.497
4,PROP000005,Koramangala,Project_4,Snn Raj,7147.0,4BHK,15.345392,Secondary,HNI,2024-12-31,...,Broker,0,5.312510,18,Too Far From My Office.,21471.096187,2024Q4,Not Booked,0,0.507


In [28]:
df.to_csv("Cleaned_Luxury_Housing_Bangalore.csv", index=False)